In [1]:
import geemap
import ee

### Add simple map 

In [2]:
Map = geemap.Map(center = [41.25827,28.928777],zoom=13)
Map.add_basemap('Esri Satellite')
Map

Enter verification code: 4/1AY0e-g5hoYq2E7iON3b28KVdNws1uaPt_YldsjukZgrJ94cbN5ig3HFezSc

Successfully saved authorization token.


Map(center=[41.25827, 28.928777], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBu…

### Add USGS Map -  Global Land Cover (2009) and  modis Land Cover (2001-2016) and Landsat 8 Data

In [4]:
dem = ee.Image("CGIAR/SRTM90_V4")
landcover_glob = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover') # Global Land Cover (2009 )
landcover_modis = ee.ImageCollection('MODIS/006/MCD12Q1').select('LC_Type1') # modis Land Cover (2001-2016)
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
vis_params = {
  'min': 0,
  'max': 1000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

igbpLandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
};

Map.addLayer(dem,vis_params,'DEM',True,0.9)
Map.addLayer(landcover_glob,{},'landcover_glob')
Map.addLayer(landcover_modis,igbpLandCoverVis,'landcover_modis')
Map.addLayer(landsat, {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 0.4}, 'landsat')


### All avaliable basemaps

In [5]:
basemaps = geemap.ee_basemaps.keys() # Basemaps
print(basemaps)

dict_keys(['ROADMAP', 'SATELLITE', 'TERRAIN', 'HYBRID', 'ESRI', 'Esri Ocean', 'Esri Satellite', 'Esri Standard', 'Esri Terrain', 'Esri Transportation', 'Esri Topo World', 'Esri National Geographic', 'Esri Shaded Relief', 'Esri Physical Map', 'FWS NWI Wetlands', 'FWS NWI Wetlands Raster', 'Google Maps', 'Google Satellite', 'Google Terrain', 'Google Satellite Hybrid', 'NLCD 2016 CONUS Land Cover', 'NLCD 2013 CONUS Land Cover', 'NLCD 2011 CONUS Land Cover', 'NLCD 2008 CONUS Land Cover', 'NLCD 2006 CONUS Land Cover', 'NLCD 2004 CONUS Land Cover', 'NLCD 2001 CONUS Land Cover', 'USGS NAIP Imagery', 'USGS Hydrography', 'USGS 3DEP Elevation', 'OpenStreetMap.Mapnik', 'OpenStreetMap.BlackAndWhite', 'OpenStreetMap.DE', 'OpenStreetMap.France', 'OpenStreetMap.HOT', 'OpenTopoMap', 'Hydda.Full', 'Hydda.Base', 'Esri.WorldStreetMap', 'Esri.DeLorme', 'Esri.WorldTopoMap', 'Esri.WorldImagery', 'Esri.NatGeoWorldMap', 'HikeBike.HikeBike', 'MtbMap', 'CartoDB.Positron', 'CartoDB.DarkMatter', 'NASAGIBS.ModisTe

### Split Map View

In [ ]:
collection = ee.ImageCollection("USGS/NLCD")
print(collection.aggregate_array('system:id').getInfo())

- USGS Land Cover Comparison

In [ ]:
nlcd_2001 = ee.Image('USGS/NLCD/NLCD2001').select('landcover')
nlcd_2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')

left_layer = geemap.ee_tile_layer(nlcd_2001, {}, 'NLCD 2001')
right_layer = geemap.ee_tile_layer(nlcd_2016, {}, 'NLCD 2016')

Map = geemap.Map()
Map.split_map(left_layer, right_layer)
Map

### Feature Drawing and clip raster geometry

In [ ]:
Map = geemap.Map()
Map

In [ ]:
# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Engine DEM to map
Map.addLayer(image, vis_params, 'SRTM DEM')

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')

In [ ]:
Map.draw_features
Map.draw_last_feature

In [ ]:
roi = ee.FeatureCollection(Map.draw_features)
selected_states = states.filterBounds(roi) # Select states that intersects drawn polygon!
Map.addLayer(selected_states, {}, "Selected states")

### Clip dem with geometry

In [ ]:
cliped_image = image.clip(selected_states)
Map.addLayer(cliped_image, vis_params, "Clipped DEM")

In [ ]:
cliped_image.getInfo()

### Add Shapefile Data

In [ ]:
import os

Map = geemap.Map(center = [41.25827,28.928777],zoom=13)
Map.add_basemap('Esri Satellite')
Map


In [ ]:
folder  = '/home/cak/Desktop/lake_extraction/Data/sulakalanlar'
ramsar_file = 'ramsar_ist.shp'
countries = geemap.shp_to_ee(os.path.join(folder,ramsar_file))
Map.addLayer(countries, {}, 'Ramsar')

### EE to Shapefile

In [ ]:
# countries_shp = '../data/countries.shp'
# countries = geemap.shp_to_ee(countries_shp)
# Map.addLayer(countries, {}, 'Countries')
# geemap.ee_to_shp(countries, filename='../data/countries_new.shp')

### Export 2 Geotif

In [ ]:
Map = geemap.Map(center = [41.25827,28.928777],zoom=13)
Map.add_basemap('Esri Satellite')
Map

In [ ]:
# if you like to draw a point
# roi = Map.draw_last_feature
# roi.getInfo()

In [ ]:
aoi = ee.Geometry.Point([28.923747, 41.257073])
aoi.getInfo()

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")


bands = ['B2','B3','B4','B5','B6','B7']

image = landsat \
            .filterBounds(aoi) \
            .sort('CLOUD_COVER') \
            .first() \
            .select(bands)

landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.6,
    'min' : 0,
    'max' : 0.3
}

Map.addLayer(image, landsat_vis, "LANDSAT/LC08/C01/T1_TOA", True, 1)

#### Download image

In [ ]:
import os
out_dir = '/home/cak/Desktop/lake_extraction/Geemap/exported'
filename = os.path.join(out_dir, 'landsat.tif')

In [ ]:
feature = Map.draw_last_feature
roi = feature.geometry()

In [ ]:
roi.getInfo()

In [ ]:
image = image.clip(roi).unmask()
geemap.ee_export_image(image, filename=filename, scale=30, region=roi, file_per_band=False)

# Spectral Linear Unmixing

In [1]:
import geemap
import ee

Map = geemap.Map(center = [41.25827,28.928777],zoom=13)
Map.add_basemap('Esri Satellite')
Map


Map(center=[41.25827, 28.928777], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBu…

In [7]:
aoi = ee.Geometry.Point([28.923747, 41.257073])
aoi.getInfo()

{'type': 'Point', 'coordinates': [28.923747, 41.257073]}

In [8]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")


bands = ['B2','B3','B4','B5','B6','B7']

image = landsat \
            .filterBounds(aoi) \
            .sort('CLOUD_COVER') \
            .first() \
            .select(bands)

#             .filterDate('2019-01-01', '2020-10-31') \ 


landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.6,
    'min' : 0,
    'max' : 0.3
}

Map.addLayer(image, landsat_vis, "LANDSAT/LC08/C01/T1_TOA", True, 1)

In [60]:
# image.getInfo()
# image.get('CLOUD_COVER').getInfo()
# image.get('system:time_start').getInfo()
# ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

## Image Stats

In [5]:
landsat_props = geemap.image_props(image)
# landsat_props.getInfo()

In [6]:
Date = landsat_props.get('IMAGE_DATE').getInfo()
CC= landsat_props.get('CLOUD_COVER').getInfo()
Zone = landsat_props.get('UTM_ZONE').getInfo()
Scale = landsat_props.get('NOMINAL_SCALE').getInfo()


In [7]:
from pprint import pprint

Stats = {
    'Date' : Date,
    'Cloud Cover' : "{:.4} %".format(float(CC)*100),
    'UTM Zone ' : Zone, 
    'Resolution' : Scale
}

pprint(Stats)

{'Cloud Cover': '1.0 %',
 'Date': '2013-07-30',
 'Resolution': 30,
 'UTM Zone ': 35}


### Add Ramsar to Map

In [13]:
ramsar_shp = '/home/cak/Desktop/lake_extraction/Data/sulakalanlar/ramsar_ist.shp'
ramsar = geemap.shp_to_ee(ramsar_shp)
Map.addLayer(ramsar, {}, 'Ramsar')

## Draw Endmembers

In [ ]:
# Map.draw_features

In [ ]:
# water_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# forest_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# soil_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# geemap.ee_to_shp(water_ee, filename='./exported/water.shp')

In [ ]:
# geemap.ee_to_shp(forest_ee, filename='./exported/forest.shp')

In [ ]:
# geemap.ee_to_shp(soil_ee, filename='./exported/soil.shp')

### Read Endmembers

In [8]:
water_shp = './exported/water.shp'
water = geemap.shp_to_ee(water_shp)
Map.addLayer(water, {}, 'Water')

forest_shp = './exported/forest.shp'
forest = geemap.shp_to_ee(forest_shp)
Map.addLayer(forest, {}, 'forest')

soil_shp = './exported/soil.shp'
soil = geemap.shp_to_ee(soil_shp)
Map.addLayer(soil, {}, 'soil')

### Endmember Zonal Statistics

In [9]:
waterMean = image.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': water.geometry().getInfo(),
        'scale': 30,
        'maxPixels': 1e9
    }).values()

forestMean = image.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': forest.geometry().getInfo(),
        'scale': 30,
        'maxPixels': 1e9
    }).values()

soilMean = image.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': soil.geometry().getInfo(),
        'scale': 30,
        'maxPixels': 1e9
    }).values()

In [10]:
endmembers = ee.Array.cat([waterMean , forestMean , soilMean] , 1)

In [11]:
arrayImage = image.toArray().toArray(1)

unmixed = ee.Image(endmembers).matrixSolve(arrayImage)

unmixedImage = unmixed.arrayProject([0]).arrayFlatten([['water','forest','soil']])

In [12]:
Map.addLayer(unmixedImage, {}, 'Unmixed')

In [14]:
waterBand = unmixedImage.select('water').gt(0.95)

Map.addLayer(waterBand, {} , 'waterBand')

In [15]:
classes = waterBand.reduceToVectors(
**{
  'reducer': ee.Reducer.countEvery(), 
  'geometry': waterBand.geometry().getInfo(), 
  'scale': 30,
  'maxPixels': 1e8
});


In [16]:
result = ee.FeatureCollection(classes);
Map.addLayer(result,{},'Water_Vector');

# Supervised classification


In [17]:
# clip image
# image_roi = image.clip(roi.geometry())
# Map.addLayer(image_roi,landsat_vis,'Cropped_Landsat')

In [24]:
waterf = ee.Feature(water.geometry(), {'class': 0, 'name': 'water'});
forestf = ee.Feature(forest.geometry(), {'class': 1, 'name': 'forest'});
soilf = ee.Feature(soil.geometry(), {'class': 2, 'name': 'soil'});

In [25]:
soilf

In [26]:
trainingFeatures = ee.FeatureCollection([waterf, forestf, soilf])

In [28]:
classifierTraining = image.select(bands).sampleRegions(
      collection= trainingFeatures, 
      properties= ['class'], 
      scale= 30
    );

In [29]:
# // Randomly split the data into 60% for training, and 40% for testing
trainingTesting = classifierTraining.randomColumn('random',111009);

training = trainingTesting.filter(ee.Filter.lt('random', 0.6));

testing = trainingTesting.filter(ee.Filter.gte('random', 0.6));

### Non-linear regression functions

In [30]:
cartclassifier = ee.Classifier.cart(randomSeed=111009).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    );

In [31]:
cartClasifficationImage = image.select(bands).classify(cartclassifier);

Map.addLayer(cartClasifficationImage, {'min': 0, 'max': 2,
                                   'palette':['blue', 'green','yellow']},'CART classification');

In [127]:
rfClassification = ee.Classifier.smileRandomForest(numberOfTrees=1, seed=111009).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    )

In [128]:
# // Perform the RF regression on the landsat image
rfClassificationImage = image.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification');

In [132]:
# ee.Classifier.libsvm(decisionProcedure, svmType, kernelType, shrinking, degree, gamma, coef0, cost, nu, terminationEpsilon, lossEpsilon, oneClass)

# // Create an SVM classifier with custom parameters.
svClassification = ee.Classifier.libsvm(kernelType='RBF',gamma=1,cost=100).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    )

In [133]:
# // Perform the RF regression on the landsat image
svClassificationImage = image.select(bands).classify(svClassification);
    
# // Visualize the RF regression
Map.addLayer(svClassificationImage,{'min': 0, 'max': 2,
                                   'palette':['blue', 'green','yellow']}, 'SV CLassification');